In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# Load datasets
df1 = pd.read_csv('/home/wahba/Documents/cicid/cicids2017/original/csv/TrafficLabelling/Wednesday-workingHours.pcap_ISCX.csv')
df2 = pd.read_csv('/home/wahba/Documents/cicid/CICFlowMeter/data/daily/2025-11-25_Flow.csv')
df3 = pd.read_csv('/home/wahba/Documents/cicid/cicids2017/generated/cicflowmeter-python/flow2.csv')
df4 = pd.read_csv('/home/wahba/Documents/nids3/tests/flow/flow_generation5.csv')

In [3]:
# Cleaning column names
col_names = {col: col.strip() for col in df1.columns}
df1.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df2.columns}
df2.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df3.columns}
df3.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df4.columns}
df4.rename(columns=col_names, inplace=True)

del col_names

In [4]:
print(df1.columns.tolist())
print(df2.columns.tolist())
print(df3.columns.tolist())
print(df4.columns.tolist())

['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH 

In [5]:
df1['Label'].value_counts()

Label
BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name: count, dtype: int64

In [28]:
# Filter out only DoS slowhttptest attacks
df1_dos = df1[df1['Label'] == 'DoS Hulk'].copy()

In [29]:
df1_dos['Protocol'] = df1_dos['Protocol'].replace({6 : 'TCP', 17: 'UDP', 1: 'ICMP'})
df2['Protocol'] = df2['Protocol'].replace({6 : 'TCP', 17: 'UDP', 1: 'ICMP'})
df3['protocol'] = df3['protocol'].replace({6 : 'TCP', 17: 'UDP', 1: 'ICMP'})

In [30]:
df1_dos['flow_key'] = (
    df1_dos['Source IP'].astype(str) + ':' +
    df1_dos['Source Port'].astype(str) + '-' +
    df1_dos['Destination IP'].astype(str) + ':' +
    df1_dos['Destination Port'].astype(str) + '-' +
    df1_dos['Protocol'].astype(str)
    )
df2['flow_key'] = (
    df2['Src IP'].astype(str) + ':' +
    df2['Src Port'].astype(str) + '-' +
    df2['Dst IP'].astype(str) + ':' +
    df2['Dst Port'].astype(str) + '-' +
    df2['Protocol'].astype(str)
    )
df3['flow_key'] = (
    df3['src_ip'].astype(str) + ':' +
    df3['src_port'].astype(str) + '-' +
    df3['dst_ip'].astype(str) + ':' +
    df3['dst_port'].astype(str) + '-' +
    df3['protocol'].astype(str)
    )

In [31]:
print(df1_dos.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(231073, 86)
(15536, 85)
(13393, 83)
(998, 53)


In [34]:
http_dos_keys = df1_dos['flow_key'].unique().tolist()
df2_dos = df2[df2['flow_key'].isin(http_dos_keys)].copy()
df3_dos = df3[df3['flow_key'].isin(http_dos_keys)].copy()
df4_dos = df4[df4['flow_key'].isin(http_dos_keys)].copy()

print(df1_dos.shape)
print(df2_dos.shape)
print(df3_dos.shape)
print(df4_dos.shape)

# # Check first few flow keys and durations
# print(df2[['flow_key', 'Timestamp', 'Flow Duration']].head(2))
# # print(df3[['flow_key', 'timestamp', 'flow_duration']].head())
# print(df4[['flow_key', 'Start Time', 'Flow Duration']].head(2))

(231073, 86)
(11445, 85)
(7605, 83)
(798, 53)


# Test model prediction on flow generated by CICFLOWMETER and from CICIDS2017

In [35]:
import joblib

rf = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
xgb = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
knn = joblib.load("/home/wahba/Documents/models_anacletu/knn_model.joblib")
robust_scaler = joblib.load("/home/wahba/Documents/models_anacletu/robust_scaler.joblib")

/home/wahba/miniconda3/envs/ml2/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/wahba/miniconda3/envs/ml2/lib/python3.12/pickle.py:1760: UserWarning: [13:57:31] WARNING: /croot/xgboost-split_1749630910898/work/src/collective/../data/../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that 

In [37]:
df3_dos.columns.tolist()

['src_ip',
 'dst_ip',
 'src_port',
 'dst_port',
 'protocol',
 'timestamp',
 'flow_duration',
 'flow_byts_s',
 'flow_pkts_s',
 'fwd_pkts_s',
 'bwd_pkts_s',
 'tot_fwd_pkts',
 'tot_bwd_pkts',
 'totlen_fwd_pkts',
 'totlen_bwd_pkts',
 'fwd_pkt_len_max',
 'fwd_pkt_len_min',
 'fwd_pkt_len_mean',
 'fwd_pkt_len_std',
 'bwd_pkt_len_max',
 'bwd_pkt_len_min',
 'bwd_pkt_len_mean',
 'bwd_pkt_len_std',
 'pkt_len_max',
 'pkt_len_min',
 'pkt_len_mean',
 'pkt_len_std',
 'pkt_len_var',
 'fwd_header_len',
 'bwd_header_len',
 'fwd_seg_size_min',
 'fwd_act_data_pkts',
 'flow_iat_mean',
 'flow_iat_max',
 'flow_iat_min',
 'flow_iat_std',
 'fwd_iat_tot',
 'fwd_iat_max',
 'fwd_iat_min',
 'fwd_iat_mean',
 'fwd_iat_std',
 'bwd_iat_tot',
 'bwd_iat_max',
 'bwd_iat_min',
 'bwd_iat_mean',
 'bwd_iat_std',
 'fwd_psh_flags',
 'bwd_psh_flags',
 'fwd_urg_flags',
 'bwd_urg_flags',
 'fin_flag_cnt',
 'syn_flag_cnt',
 'rst_flag_cnt',
 'psh_flag_cnt',
 'ack_flag_cnt',
 'urg_flag_cnt',
 'ece_flag_cnt',
 'down_up_ratio',
 'pkt_s

# Standardizing dataframes to feed to ML model

In [39]:
final_columns_df1 = {
    'Destination Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Total Fwd Packets',
    'Total Length of Fwd Packets': 'Total Length of Fwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Min Packet Length': 'Min Packet Length',
    'Max Packet Length': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'Average Packet Size': 'Average Packet Size',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Init_Win_bytes_forward': 'Init_Win_bytes_forward',
    'Init_Win_bytes_backward': 'Init_Win_bytes_backward',
    'act_data_pkt_fwd': 'act_data_pkt_fwd',
    'min_seg_size_forward': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

final_columns_df2 = {
    'Dst Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packet': 'Total Fwd Packets',
    'Total Length of Fwd Packet': 'Total Length of Fwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Packet Length Min': 'Min Packet Length',
    'Packet Length Max': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'Average Packet Size': 'Average Packet Size',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'FWD Init Win Bytes': 'Init_Win_bytes_forward',
    'Bwd Init Win Bytes': 'Init_Win_bytes_backward',
    'Fwd Act Data Pkts': 'act_data_pkt_fwd',
    'Fwd Seg Size Min': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

final_columns_df3 = {
    'dst_port': 'Destination Port',
    'flow_duration': 'Flow Duration',
    'tot_fwd_pkts': 'Total Fwd Packets',
    'totlen_fwd_pkts': 'Total Length of Fwd Packets',
    'fwd_pkt_len_max': 'Fwd Packet Length Max',
    'fwd_pkt_len_min': 'Fwd Packet Length Min',
    'fwd_pkt_len_mean': 'Fwd Packet Length Mean',
    'fwd_pkt_len_std': 'Fwd Packet Length Std',
    'bwd_pkt_len_max': 'Bwd Packet Length Max',
    'bwd_pkt_len_min': 'Bwd Packet Length Min',
    'bwd_pkt_len_mean': 'Bwd Packet Length Mean',
    'bwd_pkt_len_std': 'Bwd Packet Length Std',
    'flow_byts_s': 'Flow Bytes/s',
    'flow_pkts_s': 'Flow Packets/s',
    'flow_iat_mean': 'Flow IAT Mean',
    'flow_iat_std': 'Flow IAT Std',
    'flow_iat_max': 'Flow IAT Max',
    'flow_iat_min': 'Flow IAT Min',
    'fwd_iat_tot': 'Fwd IAT Total',
    'fwd_iat_mean': 'Fwd IAT Mean',
    'fwd_iat_std': 'Fwd IAT Std',
    'fwd_iat_max': 'Fwd IAT Max',
    'fwd_iat_min': 'Fwd IAT Min',
    'bwd_iat_tot': 'Bwd IAT Total',
    'bwd_iat_mean': 'Bwd IAT Mean',
    'bwd_iat_std': 'Bwd IAT Std',
    'bwd_iat_max': 'Bwd IAT Max',
    'bwd_iat_min': 'Bwd IAT Min',
    'fwd_header_len': 'Fwd Header Length',
    'bwd_header_len': 'Bwd Header Length',
    'fwd_pkts_s': 'Fwd Packets/s',
    'bwd_pkts_s': 'Bwd Packets/s',
    'pkt_len_min': 'Min Packet Length',
    'pkt_len_max': 'Max Packet Length',
    'pkt_len_mean': 'Packet Length Mean',
    'pkt_len_std': 'Packet Length Std',
    'pkt_len_var': 'Packet Length Variance',
    'fin_flag_cnt': 'FIN Flag Count',
    'psh_flag_cnt': 'PSH Flag Count',
    'ack_flag_cnt': 'ACK Flag Count',
    'pkt_size_avg': 'Average Packet Size',
    'subflow_fwd_byts': 'Subflow Fwd Bytes',
    'init_fwd_win_byts': 'Init_Win_bytes_forward',
    'init_bwd_win_byts': 'Init_Win_bytes_backward',
    'fwd_act_data_pkts': 'act_data_pkt_fwd',
    'fwd_seg_size_min': 'min_seg_size_forward',
    'active_mean': 'Active Mean',
    'active_max': 'Active Max',
    'active_min': 'Active Min',
    'idle_mean': 'Idle Mean',
    'idle_max': 'Idle Max',
    'idle_min': 'Idle Min'
}


In [52]:
df3_dos.columns.tolist()

['src_ip',
 'dst_ip',
 'src_port',
 'dst_port',
 'protocol',
 'timestamp',
 'flow_duration',
 'flow_byts_s',
 'flow_pkts_s',
 'fwd_pkts_s',
 'bwd_pkts_s',
 'tot_fwd_pkts',
 'tot_bwd_pkts',
 'totlen_fwd_pkts',
 'totlen_bwd_pkts',
 'fwd_pkt_len_max',
 'fwd_pkt_len_min',
 'fwd_pkt_len_mean',
 'fwd_pkt_len_std',
 'bwd_pkt_len_max',
 'bwd_pkt_len_min',
 'bwd_pkt_len_mean',
 'bwd_pkt_len_std',
 'pkt_len_max',
 'pkt_len_min',
 'pkt_len_mean',
 'pkt_len_std',
 'pkt_len_var',
 'fwd_header_len',
 'bwd_header_len',
 'fwd_seg_size_min',
 'fwd_act_data_pkts',
 'flow_iat_mean',
 'flow_iat_max',
 'flow_iat_min',
 'flow_iat_std',
 'fwd_iat_tot',
 'fwd_iat_max',
 'fwd_iat_min',
 'fwd_iat_mean',
 'fwd_iat_std',
 'bwd_iat_tot',
 'bwd_iat_max',
 'bwd_iat_min',
 'bwd_iat_mean',
 'bwd_iat_std',
 'fwd_psh_flags',
 'bwd_psh_flags',
 'fwd_urg_flags',
 'bwd_urg_flags',
 'fin_flag_cnt',
 'syn_flag_cnt',
 'rst_flag_cnt',
 'psh_flag_cnt',
 'ack_flag_cnt',
 'urg_flag_cnt',
 'ece_flag_cnt',
 'down_up_ratio',
 'pkt_s

In [53]:
df1_dos_standardized = df1_dos[list(final_columns_df1.keys())].rename(columns=final_columns_df1)
df2_dos_standardized = df2_dos[list(final_columns_df2.keys())].rename(columns=final_columns_df2)
df3_dos_standardized = df3_dos[list(final_columns_df3.keys())].rename(columns=final_columns_df3)
print("expected number of features for the model:", len(final_columns_df1))

expected number of features for the model: 52


In [54]:
print("df1_dos_standardized shape:", df1_dos_standardized.shape)
print("df2_dos_standardized shape:",df2_dos_standardized.shape)
print("df3_dos_standardized shape:", df3_dos_standardized.shape)
print("Expected number of features for the model:", len(final_columns_df1))

df1_dos_standardized shape: (231073, 52)
df2_dos_standardized shape: (11445, 52)
df3_dos_standardized shape: (7605, 52)
Expected number of features for the model: 52


In [55]:
print(df2_dos_standardized.columns.tolist())
print(df3_dos_standardized.columns.tolist())

['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Length of Fwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'Average Packet Size', 'Subflow Fwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Max', 'Id

In [56]:
# rf_predictions_df1 = pd.Series(rf.predict(df1_dos_standardized))
# xgb_predictions_df1 = pd.Series(xgb.predict(df1_dos_standardized))
# knn_predictions_df1 = pd.Series(knn.predict(robust_scaler.transform(df1_dos_standardized)))

rf_predictions_df2 = pd.Series(rf.predict(df2_dos_standardized))
xgb_predictions_df2 = pd.Series(xgb.predict(df2_dos_standardized))
knn_predictions_df2 = pd.Series(knn.predict(robust_scaler.transform(df2_dos_standardized)))

rf_predictions_df3 = pd.Series(rf.predict(df3_dos_standardized))
xgb_predictions_df3 = pd.Series(xgb.predict(df3_dos_standardized))
knn_predictions_df3 = pd.Series(knn.predict(robust_scaler.transform(df3_dos_standardized)))

In [57]:
# print("rf preditions df1\n",rf_predictions_df1.value_counts(), "\n")
# print("xgb preditions df1\n",xgb_predictions_df1.value_counts(), "\n")
# print("knn preditions df1\n",knn_predictions_df1.value_counts(), "\n")
print("rf preditions df2\n",rf_predictions_df2.value_counts(), "\n")
print("xgb preditions df2\n",xgb_predictions_df2.value_counts(), "\n")
print("knn preditions df2\n",knn_predictions_df2.value_counts(), "\n")
print("rf preditions df3\n",rf_predictions_df3.value_counts(), "\n")
print("xgb preditions df3\n",xgb_predictions_df3.value_counts(), "\n")
print("knn preditions df3\n",knn_predictions_df3.value_counts(), "\n")

rf preditions df2
 1    9543
0    1902
Name: count, dtype: int64 

xgb preditions df2
 1    9543
0    1902
Name: count, dtype: int64 

knn preditions df2
 Normal Traffic    5792
DoS               5191
DDoS               356
Brute Force         54
Port Scanning       49
Web Attacks          3
Name: count, dtype: int64 

rf preditions df3
 0    6431
1    1174
Name: count, dtype: int64 

xgb preditions df3
 0    6431
1    1174
Name: count, dtype: int64 

knn preditions df3
 Normal Traffic    5894
DDoS               908
DoS                648
Port Scanning      155
Name: count, dtype: int64 

